In [9]:
import json
import pandas as pd

In [4]:
with open('../Ecommerce_FAQ_Chatbot_dataset.json') as json_file:
    data = json.load(json_file)

In [7]:
print(data['questions'][0])

{'question': 'How can I create an account?', 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."}


In [8]:
with open('dataset.json', 'w') as f:
    json.dump(data['questions'], f)

In [10]:
pd.DataFrame(data['questions']).head()

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


In [11]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/falcon_llm/llm_env/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/libbitsandbytes_cpu.so
False
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Required library version not found: libbitsandbytes_cpu.so. Maybe you need to compile it from source?
CUDA SETUP: Defaulting to libbitsandbytes_cpu.so...

================================================ERROR=====================================
CUDA SETUP: CUDA detection failed! Possible reasons:
1. CUDA driver not installed
2. CUDA not installed
3. You have multiple conf

/home/ubuntu/falcon_llm/llm_env/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/ubuntu/falcon_llm/llm_env/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/ubuntu/falcon_llm/llm_env/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/ubuntu/falcon_llm/llm_env/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.p

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [1]:
import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, concatenate_datasets, Dataset

In [2]:
ds = load_dataset("OpenAssistant/oasst1")
# lets convert the train dataset to a pandas df
df = ds["train"].to_pandas()

Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-2960c57d7e52ab15/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 314.60it/s]


In [3]:
df.head()

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
0,6ab24d72-0181-4594-a9cd-deaf170242fb,None,c3fe8c76-fc30-4fa7-b7f8-c492f5967d18,2023-02-05T14:23:50.983374+00:00,Can you write a short introduction about the r...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.00044308538781479, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,6ab24d72-0181-4594-a9cd-deaf170242fb,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,True,False,0.0,False,None,"{'toxicity': 0.00026396565954200923, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_labeling'], 'count': [3...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
2,6708c47f-05c9-4346-b3d2-40b2bd24fde4,c8e83833-ecbc-44fe-b6db-735228c25a1c,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T18:48:49.391686+00:00,Now explain it to a dog,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.03648477792739868, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,None,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
3,343ee2d4-87ae-41fd-a768-bdd65959dc4a,6ab24d72-0181-4594-a9cd-deaf170242fb,49ddcb0d-6588-43bd-858d-19c577f12e7b,2023-02-06T13:37:56.044680+00:00,Monopsony is a market structure in which there...,assistant,en,3,True,False,1.0,False,None,"{'toxicity': 0.0008866374846547842, 'severe_to...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_labeling...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
4,18145bf4-37fd-4ac0-80f5-6108b5f2b365,343ee2d4-87ae-41fd-a768-bdd65959dc4a,e10e99a0-38ac-4b07-bf5d-4427696e4e0d,2023-02-06T18:52:51.428543+00:00,How can one fight back when a monospony had be...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.0009362137061543763, 'severe_to...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1'], 'count': [1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


In [4]:
message_tree_ids = np.unique(np.array(df["message_tree_id"]))
message_tree_ids


array(['001199c7-7135-4e64-bee9-48ea862243b4',
       '00161eaf-7262-426c-8db3-7eb447679840',
       '00258783-e10f-4d90-a3a4-abd22ea903f8', ...,
       'ffe9aca3-a201-4803-a061-c17f803d0046',
       'fff90ee9-d91a-42d3-8a05-ce3cd68109e9',
       'fff9ef1b-dc66-44d0-8790-92b02fe2c571'], dtype=object)

In [5]:
messages = {}
messages['message_tree_id'] = []
messages['message_tree_text'] = []

In [6]:

for message_tree_id in message_tree_ids:
    try:

        one_message_tree = df.query(f"message_tree_id == '{message_tree_id}'").sort_values("created_date")
        text = ""
    

        text += "<human>: " + one_message_tree.iloc[0].text

        children = one_message_tree[one_message_tree.parent_id == one_message_tree.iloc[0].message_id]

        child = children[children['rank'] == 0.0]
        text += '\n' + "<bot>: " + child.iloc[0].text
    

        flag=True
        while flag:
            try:

                children = one_message_tree[one_message_tree.parent_id == child.message_id.iloc[0]]
                children.index
                one_message_tree.loc[children.index].iloc[0].role
                text += '\n' + "<human>: " + one_message_tree.loc[children.index].iloc[0].text
    

                children = one_message_tree[one_message_tree.parent_id == one_message_tree.loc[children.index].iloc[0].message_id]
                children

                child = children[children['rank'] == 0.0]
                text += '\n' + "<bot>: " + child.iloc[0].text
            except:
                flag=False
    
        messages['message_tree_id'].append(message_tree_id)
        messages['message_tree_text'].append(text)

    except IndexError:
        pass

message_df = pd.DataFrame.from_dict(messages)



data = Dataset.from_pandas(message_df)
data


Dataset({
    features: ['message_tree_id', 'message_tree_text'],
    num_rows: 9823
})

In [10]:
message_df['message_tree_text'][0]

"<human>: Do you have any information about the Commodore 64?\n<bot>: Of course. The Commodore 64 is a 8-bit home computer that was released by Commodore International in August 1982. It is one of the most successful home computers of all time, with over 17 million units sold. It featured 64 kilobytes of RAM, color graphics, and the SID sound chip. It was widely used for gaming, programming, and home productivity applications. It was succeeded by the Commodore 128.\n\nThere is still a thriving community of Commodore 64 enthusiasts around the world who continue to enjoy playing and programming on the system. You can find more information about the Commodore 64 on various websites and forums dedicated to the system.\n\nIf you have any specific questions about the Commodore 64's hardware, software, or programming languages, please feel free to ask."